In [ ]:

import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier
import xgboost as xgb 
import os

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
# !pip install catboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 7.9 MB/s eta 0:00:00


In [ ]:
def import_data_set(file_name:str):
    return pd.read_csv(file_name)

In [ ]:
def fill_data_with_0(data:pd.DataFrame()):
    return data.fillna(0)

In [ ]:
def split_data_into_X_and_y(train_data:pd.DataFrame(),label:str):
    X = train_data.drop(label,axis=1)
    y = train_data[label]
    return X,y

In [ ]:
os.chdir("/content/drive/MyDrive/Toxicity Prediction")

In [ ]:
train_data = import_data_set("train_data.csv")
test_data = import_data_set("test_data.csv")

In [ ]:
train_data = fill_data_with_0(train_data)
test_data = fill_data_with_0(test_data)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X , y = split_data_into_X_and_y(train_data, "label")


In [ ]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

In [ ]:
import lightgbm as lgb
import xgboost as xgb 

clf1 = lgb.LGBMClassifier(random_state=60,n_estimators=1900,learning_rate=0.2)
clf2 = lgb.LGBMClassifier(random_state=60,n_estimators=2100,learning_rate=0.2)
clf3 = CatBoostClassifier(iterations=25000, learning_rate=0.1, depth=6, l2_leaf_reg=3, loss_function='Logloss', random_seed=60)
clf4 = xgb.XGBClassifier(random_state=60,n_estimators=700,eta=0.35)

# clf1 = lgb.LGBMClassifier(random_state=60,n_estimators=1800,learning_rate=0.2)
# clf2 = xgb.XGBClassifier(random_state=60,n_estimators=700,eta=0.35)
# clf3 = xgb.XGBClassifier(random_state=60,n_estimators=700,eta=0.35)
# clf4 = CatBoostClassifier(iterations=25000, learning_rate=0.1, depth=6, l2_leaf_reg=3, loss_function='Logloss', random_seed=60)



voting_clf = VotingClassifier(estimators=[('lgm', clf1), ('cat', clf2), ('xgb', clf3), ('xgb1', clf3)], voting='soft')


In [ ]:
voting_clf.fit(X,y)

VotingClassifier(estimators=[('lgm',
                              LGBMClassifier(learning_rate=0.2,
                                             n_estimators=1800,
                                             random_state=60)),
                             ('cat',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False, eta=0.35,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_i...
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=700, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, ...))],
                 voting='soft')

In [ ]:
predict = encoder.inverse_transform(voting_clf.predict(test_data))

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.25,shuffle=True)

predict2 = voting_clf.predict(X_test)

In [ ]:
submission_file = pd.read_csv("./sample_submission.csv")
submission_file.Predicted = predict

In [ ]:
print(np.count_nonzero(predict==1))
print(np.count_nonzero(predict==2))

1317
9677


In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, predict2, average=None).mean()

0.9899001723115339

In [ ]:
submission_file.to_csv("submission_13.csv",index=False)